In [12]:
import numpy as np
import cv2
import glob

In [19]:
#escolhe o método de extração de keypoints
feature_extractor = 'sift'

#são lidas as imagens
images = [cv2.imread(file) for 
          file in sorted(glob.glob("caminho_das_imagens/*.jpg"))]

#diminui-se a resolução das imagens para que o tempo de execução seja menor
images = [cv2.resize(images[i], (int(images[i].shape[1]*0.5), int(images[i].shape[0]*0.5))) 
          for i in range(len(images))]

In [14]:
#cria o objeto que opera o algoritmo de detecção de keypoints
def detectAndDescribe (image, method=None):
        
    if method == 'sift':
        descriptor = cv2.SIFT_create()
    elif method == 'orb':
        descriptor = cv2.ORB_create()
    elif method == 'brisk':
        descriptor = cv2.BRISK_create()
        
    (kps, features) = descriptor.detectAndCompute(image, None)
    
    return (kps, features)

#cria o objeto que corresponde os keypoints de duas imagens
def createMatcher(method, crossCheck):
    if method == 'sift':
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=crossCheck)
    else:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=crossCheck)
    
    return bf

#realiza a correspondência de pontos
def matchKeypointsKNN(featuresA, featuresB, ratio, method):
    bf = createMatcher(method, crossCheck=False)
    
    rawMatches = bf.knnMatch(featuresA, featuresB, 2)
    matches = []
    
    for m,n in rawMatches:
        if m.distance < n.distance * ratio:
            matches.append(m)
    return matches  

#opera o grafo e encontra o caminho que será percorrido
def breadth(grafo, inicio):
    visited = [False]*len(grafo)
    
    final = []
    
    queue = []
    
    queue.append(inicio)
    visited[inicio] = True
    
    while queue:
        inicio = queue.pop(0)
        print(inicio, end=" ")
        final.append(inicio)
        
        for i in grafo[inicio]:
            if visited[i] == False:
                queue.append(i)
                visited[i] = True
                
    return final
    
#encontra a homografia que sobrepõe a imagem B na imagem A
def getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh):
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    
    if len(matches) > 4:

        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])
        
        (H, status) = cv2.findHomography(ptsB, ptsA, cv2.RANSAC, reprojThresh)

        return H
    else:
        return None
    
#encontra a máscara de uma imagem
def get_mask(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = cv2.threshold(gray, 0, 1, cv2.THRESH_BINARY)[1]
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask = mask.astype(np.float32)
    mask = 1 - mask
    
    return mask

#realiza o blending de duas imagens
def merge(imgA, imgB, mask):
    maskA = 1 - mask
    
    fim = imgA * mask + imgB * maskA
        
    return fim

In [15]:
threshold = 150 #quantidade mínima de matches para que dois vértices se liguem via aresta no grafo

grafo = {}

infoImg = {}

matching = {}

nbr_imgs = len(images)

for i in range(nbr_imgs):
    grafo[i] = [] #inicializa o grafo, criando sua lista de adjacências vazia

for i in range(0, nbr_imgs):
    for j in range(0, nbr_imgs):
        imgA = images[i]
        imgB = images[j]
        
        kpsA, featuresA = detectAndDescribe(imgA, method=feature_extractor)
        kpsB, featuresB = detectAndDescribe(imgB, method=feature_extractor)
        
        infoImg[i] = (kpsA, featuresA)
        infoImg[j] = (kpsB, featuresB)

        matches = matchKeypointsKNN(featuresA, featuresB, ratio=0.3, method=feature_extractor)
            
        if len(matches) > threshold and i!=j: #cria a aresta entre vértices caso cumpram a condição
            grafo[i].append(j)
            matching[(i, j)] = matches
        
escolhaArray = [len(grafo[i]) for i in range(len(images))]

escolha = escolhaArray.index(max(escolhaArray))

percurso = breadth(grafo, escolha) #ordem de colagem das imagens 

2 0 1 3 4 5 

In [16]:
usados = []
homografias = {} #guarda as homografias entre imagens
caminhos = {} #caminho que cada vértice tem que fazer para chegar na imagem central

for i in range(len(percurso)):
    caminhos[percurso[i]] = []

for i in range(len(percurso)):
    for j in range(len(grafo[percurso[i]])):
        if  grafo[percurso[i]][j] not in usados:
            
            infoA = infoImg[percurso[i]]
            infoB = infoImg[grafo[percurso[i]][j]]

            (kpsA, featuresA) = infoA
            (kpsB, featuresB) = infoB

            matches = matching[(percurso[i], grafo[percurso[i]][j])]

            H = getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh = 4)
            
            homografias[(percurso[i], grafo[percurso[i]][j])] = H #salva as homografias
            
            caminhos[grafo[percurso[i]][j]].append(percurso[i])
            
            for k in range(len(caminhos[percurso[i]])):
                caminhos[grafo[percurso[i]][j]].append(caminhos[percurso[i]][k])
            
            usados.append(grafo[percurso[i]][j])
    
    usados.append(percurso[i])

In [7]:
#encontra a resolução final da imagem

xs = []
ys = []
finais = {}

imagesCopy = images.copy()

for i in range(len(percurso)):
    w = imagesCopy[percurso[i]].shape[1]
    h = imagesCopy[percurso[i]].shape[0]
    
    ordemAc = caminhos[percurso[i]].copy()
    ordemAc.reverse()
    ordemAc.append(percurso[i])
    
    final = np.identity(3)
    
    if len(ordemAc) > 1:
        for j in range(len(ordemAc) - 1):
            sup = homografias[(ordemAc[j], ordemAc[j+1])]
            final = np.dot(final, sup)
    
    finais[percurso[i]] = final
    
    p1 = [0, 0, 1]
    p2 = [w, 0, 1]
    p3 = [0, h, 1]
    p4 = [w, h, 1]
    p = [p1, p2, p3, p4]
    
    for pi in p:
        res = np.dot(final, pi)
        xs.append(res[0])
        ys.append(res[1])
    
width = int(max(xs) - min(xs)) + 1
heigth = int(max(ys) - min(ys)) + 1

print(width, heigth)

3850 2821


In [17]:
translate = np.array([
    [1, 0, -min(xs)],
    [0, 1, -min(ys)],
    [0, 0, 1]
])

#esta matriz será aplicada em cada uma das imagens, para levá-las ao centro do canvas

to_blend = []

#transforma cada imagem, deixando elas no seu lugar final do mosaico

for i in range(len(percurso)):
    img = imagesCopy[percurso[i]]
    
    final = translate.copy()
            
    final = np.dot(final, finais[percurso[i]])
    
    transformed = cv2.warpPerspective(img, final, (width, heigth))
    cv2.imwrite("transformado_{}.png".format(i), transformed)
    
    to_blend.append(transformed)

In [18]:
blending = to_blend.copy()

#realiza o blending com todas as imagens

for i in range(1, len(to_blend)):
    imgA = blending[i-1]
    imgB = blending[i]
        
    mask = get_mask(imgB)
    
    resultado = merge(imgA, imgB, mask)
        
    blending[i] = resultado
    
cv2.imwrite("resultado_final1.png", resultado)

True